In [1]:
pip install refractio[snowflake]

     |████████████████████████████████| 12.3MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 77.8MB/s eta 0:00:01
     |████████████████████████████████| 19.3MB 78.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 59.0MB/s eta 0:00:01
     |████████████████████████████████| 235kB 77.6MB/s eta 0:00:01
     |████████████████████████████████| 348kB 79.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 18.0MB/s eta 0:00:01
     |████████████████████████████████| 450kB 78.3MB/s eta 0:00:01
     |████████████████████████████████| 174kB 81.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 18.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 22.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 76.4MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 68.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 81.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.1MB/s eta 0:00:0

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from refractio import snowflake
import matplotlib.pyplot as plt
import pandas as pd
# To read a specific dataset published from a snowflake connection.
dataset = snowflake.get_dataframe("ORDER_INVENTORY")
dataset.columns

ModuleNotFoundError: No module named 'refractio'